In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'pandas'

In [1]:
pip install tensorflow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Load and preprocess images
images = []
annotations = []

# Adjust the path based on your actual dataset location
dataset_path = "/kaggle/input/flowers-dataset"

# Loop through directories in the dataset path
for flower_class in os.listdir(os.path.join(dataset_path, 'train')):
    class_path = os.path.join(dataset_path, 'train', flower_class)
    
    # Check if the path is a directory
    if os.path.isdir(class_path):
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            
            # Check if the path is a file and ends with a common image extension
            if os.path.isfile(image_path) and image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image = cv2.imread(image_path)
                image = cv2.resize(image, (256, 256))  # Resize to 256x256

                images.append(image)
                # Placeholder for annotation (center of the image)
                annotations.append((128, 128))  # Assuming the size is 256x256

images = np.array(images) / 255.0  # Normalize pixel values
annotations = np.array(annotations)

# Split the dataset into training, validation, and testing sets
train_images, test_images, train_annotations, test_annotations = train_test_split(
    images, annotations, test_size=0.2, random_state=42
)

# Build a simple model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(2, activation='linear'))  # Output layer for (x, y) coordinates

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_images, train_annotations, epochs=10, validation_split=0.2)

# Save the model in the native Keras format
model.save("/kaggle/working/flowers_coordinate_model", save_format="tf")

# Load the saved model
loaded_model = tf.keras.models.load_model("/kaggle/working/flowers_coordinate_model")

# Evaluate the model on the test set
test_loss = loaded_model.evaluate(test_images, test_annotations)
print("Test Loss:", test_loss)

# Make predictions on the test set
test_predictions = loaded_model.predict(test_images)

# Display a few examples
for i in range(5):
    print("Actual Coordinates:", test_annotations[i])
    print("Predicted Coordinates:", test_predictions[i])
    print()


2024-01-31 13:41:47.673827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 13:41:47.673950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 13:41:47.850752: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/10
55/55 [==============================] - 54s 964ms/step - loss: 1479.7306 - val_loss: 303.5298
Epoch 2/10
55/55 [==============================] - 75s 1s/step - loss: 207.8383 - val_loss: 128.5223
Epoch 3/10
55/55 [==============================] - 48s 876ms/step - loss: 81.3674 - val_loss: 56.1789
Epoch 4/10
55/55 [==============================] - 48s 867ms/step - loss: 38.9363 - val_loss: 39.2371
Epoch 5/10
55/55 [==============================] - 48s 878ms/step - loss: 13.1499 - val_loss: 9.7668
Epoch 6/10
55/55 [==============================] - 48s 870ms/step - loss: 7.9983 - val_loss: 23.1068
Epoch 7/10
55/55 [==============================] - 48s 879ms/step - loss: 6.3670 - val_loss: 6.0606
Epoch 8/10
55/55 [==============================] - 48s 868ms/step - loss: 3.3598 - val_loss: 6.0623
Epoch 9/10
55/55 [==============================] - 50s 903ms/step - loss: 2.2826 - val_loss: 3.3314
Epoch 10/10
18/18 [==============================] - 5s 251ms/step - loss: 3.15